In [384]:
import pandas as pd
import gurobipy as gp
import math
import pickle as pkl
from datetime import datetime, timedelta

In [385]:
product_type = "ambient"
num_time_periods = 365

In [386]:
article_data = pd.read_csv("./data/article.csv")
article_data = article_data[(article_data['TEMPERATURE_ZONE'] == product_type)]
#article_data = article_data.head(20)
# article_data = article_data[(article_data['ARTICLE_ID'] == test_article)]

In [387]:

if(product_type=="frozen"):
    warehouse_volume = 50
if(product_type=="chilled"):
    warehouse_volume = 250
if(product_type=="ambient"):
    warehouse_volume = 700
buffer_cost = 25
max_order_exceed_multiplier = 1.5

In [388]:
def createParameterMatrix(data, columns):
    parameters = []
    for column in columns:
        parameters.append(data[column].to_list())
    parameters = list(map(list, zip(*parameters)))
    return parameters

In [389]:
articles = article_data['ARTICLE_ID'].to_list()

parameters = createParameterMatrix(
    article_data,
    [
        'TEMPERATURE_ZONE',
        'VOLUME_M3_PER_CU',
        'MEAN_SHELF_LIFE',
        'CU_PER_TU',
        'ORDERING_COST_FIXED',
        'ORDERING_COST_PER_TU',
        'CLEARING_COST_PER_CU',
        'MINIMUM_ORDER_QUANTITY_TU',
        'MAXIMUM_ORDER_QUANTITY_TU'
    ]
)
parameters_dict = dict(zip(articles, parameters))

In [390]:
items, category, volume_per_cu, shelf_life, cu_per_tu, ordering_cost_fixed, ordering_cost_per_tu, clearing_cost_per_cu, minimum_order_quantity_tu, maximum_order_quantity_tu = gp.multidict(parameters_dict)

In [391]:
forecast_data = pd.read_csv('./data/sales_'+str(num_time_periods)+'.csv')
forecast_data = forecast_data[forecast_data['ARTICLE_ID'].isin(articles)]

In [392]:
# Create a new dataframe with all dates
all_dates_df = pd.DataFrame({'DATE': pd.date_range(start=min(forecast_data['DATE']), end=max(forecast_data['DATE']), freq='D')}).astype(str)

# Group the original dataframe by item
grouped = forecast_data.groupby('ARTICLE_ID')

# Initialize an empty list to store the new dataframes
new_dfs = []

# Loop over each group
for item, group_df in grouped:
    
    group_df['DATE'] = pd.to_datetime(group_df['DATE']).astype(str)

    # Merge the group dataframe with the all_dates dataframe
    merged_df = pd.merge(all_dates_df, group_df, on='DATE', how='outer')
    merged_df['ARTICLE_ID'] = item
    
    # Fill in missing values
    merged_df['PICKING_QUANTITY_CU'] = merged_df['PICKING_QUANTITY_CU'].fillna(0)
    
    # Sort by date and append to the list
    new_dfs.append(merged_df.sort_values('DATE'))
    
# Concatenate all new dataframes into a single dataframe
forecast_data = pd.concat(new_dfs)
time_periods = forecast_data['DATE'].unique()

time_indexes = [*range(len(time_periods))]
date_to_index = {time_periods[i]:[*range(len(time_periods))][i] for i in time_indexes}
index_to_date = {[*range(len(time_periods))][i]:time_periods[i] for i in time_indexes}

# demand = forecast_data.groupby('DATE').apply(lambda x: dict(zip(x['ARTICLE_ID'], x['PICKING_QUANTITY_CU']))).to_dict()
# demand = dict((date_to_index[key],value) for (key,value) in demand.items())

demand = forecast_data.groupby('ARTICLE_ID').apply(lambda x: dict(zip(x['DATE'], x['PICKING_QUANTITY_CU']))).to_dict()
for item in demand.keys():
    demand[item] = dict((date_to_index[key], value) for (key, value) in demand[item].items())

In [393]:
cost_dict = {
    'fixed':0,
    'per_tu':0,
    'clearance':0,
    'buffer':0,
    'max_order_penalty':0,
    'total':0
}

In [394]:
def getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost):
        cost = 0
        if (orders[order_date]* cu_per_tu_item) - cum_demand > 0:
            cost = ((orders[order_date] * cu_per_tu_item) - cum_demand) * disposal_cost
        return cost   

def getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_item):
    max_order = 0
    if orders[order_date] > maximum_order_quantity_item: 
        #max_order = ((orders[order_date] * cu_per_tu_item) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost
        max_order = ((orders[order_date]) - maximum_order_quantity_item) * max_order_exceed_multiplier * variable_cost 
    return max_order 

def getTotalCosts(orders, order_date, cu_per_tu_item, order_q, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, cum_demand):
    tot_disposal = getCost(orders, order_date, cu_per_tu_item, cum_demand, disposal_cost)
    tot_ordering = orders[order_date] * variable_cost
    tot_fixed = fixed_cost
    tot_max_order =  getMaxOrderPenalty(orders, order_date, cu_per_tu_item, variable_cost, maximum_order_quantity_tu_item)
    return tot_disposal, tot_ordering, tot_fixed, tot_max_order

def returnExcess(orders, order_date, cu_per_tu_item, cum_demand):
    x = 0
    if order_date in orders:
        if (orders[order_date] * cu_per_tu_item) - cum_demand > 0:
            x = (orders[order_date] * cu_per_tu_item) - cum_demand 
    return x


In [395]:
def silverMeal(item):
    print(item)
    item_demand = list(demand[item].values())
    fixed_cost = ordering_cost_fixed[item]
    variable_cost = ordering_cost_per_tu[item]
    cu_per_tu_item = cu_per_tu[item]
    disposal_cost = clearing_cost_per_cu[item]
    maximum_order_quantity_tu_item = maximum_order_quantity_tu[item]
    life = shelf_life[item]
    cost = 0
    tot_disposal = 0
    tot_fixed = 0
    tot_ordering = 0
    tot_max_order = 0
    penalty = 0
    tot_excess = 0
    orders = {}
    order_date = 0 
    run_complete = False
    obj_val = 0
    while(not run_complete):
        if (order_date %7 == 6):
            order_date = order_date + 1 
            continue   
        
        min_cost = float('inf')
        cum_demand = 0
        

        for t in range(order_date, len(item_demand)):
            if(t%7==6 ):
                continue
            cum_demand += item_demand[t]
            
            if(t==order_date+life):
                order_q = cum_demand
                orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                # print(    "old tot_ordering1" , orders[order_date] * variable_cost, orders[order_date])

                #tot_excess += returnExcess(orders, order_date, cu_per_tu_item, cum_demand)
                v = (math.ceil(cum_demand / cu_per_tu_item) * cu_per_tu_item) - cum_demand
                if v>0 and cum_demand - v >0:
                    order_q = cum_demand - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                    # print(    "new tot_ordering1" , orders[order_date] * variable_cost, orders[order_date])

                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, order_q, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, cum_demand)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                order_date = t + 1
                break

            cost = fixed_cost + variable_cost * math.ceil(cum_demand/cu_per_tu_item) 
            if(math.ceil(cum_demand/cu_per_tu_item) > maximum_order_quantity_tu_item):
                penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier * variable_cost
                #penalty = (math.ceil(cum_demand/cu_per_tu_item) - maximum_order_quantity_tu_item) * max_order_exceed_multiplier  
            avg_cost = (cost + penalty) / (t-order_date+1)
            
            if(avg_cost <= min_cost):
                min_cost = avg_cost
                if(t==len(item_demand)-1):
                    order_q = cum_demand
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                    # print(    "old tot_ordering2" , orders[order_date] * variable_cost, orders[order_date])
                    #tot_excess += returnExcess(orders, order_date, cu_per_tu_item, cum_demand)
                    v = (math.ceil(cum_demand / cu_per_tu_item) * cu_per_tu_item) - cum_demand

                    #if tot_excess!=0 and (cum_demand - tot_excess > 0):
                    if v>0 and cum_demand - v >0:
                        order_q = cum_demand - v
                        orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                        # print(    "new tot_ordering2" , orders[order_date] * variable_cost, orders[order_date])
                        print(v)


                    d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, order_q, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, cum_demand)
                    tot_disposal += d
                    tot_ordering += o
                    tot_fixed += f
                    tot_max_order += m
                    run_complete = True
                    break

            else:
                order_q = cum_demand - item_demand[t]
                orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                # print(    "old tot_ordering3" , orders[order_date] * variable_cost, orders[order_date])

                #tot_excess += returnExcess(orders, order_date, cu_per_tu_item, cum_demand- item_demand[t])
                v = (math.ceil((cum_demand - item_demand[t]) / cu_per_tu_item) * cu_per_tu_item) - cum_demand

                if v>0 and cum_demand - v >0:
                    order_q = cum_demand - item_demand[t] - v
                    orders[order_date] = math.ceil(order_q/cu_per_tu_item)
                    # print(  "new tot_ordering3" , orders[order_date] * variable_cost, orders[order_date])

                d, o, f, m = getTotalCosts(orders, order_date, cu_per_tu_item, order_q, disposal_cost, variable_cost, maximum_order_quantity_tu_item, fixed_cost, cum_demand)
                tot_disposal += d
                tot_ordering += o
                tot_fixed += f
                tot_max_order += m
                order_date = t
                break
       
        obj_val += cost
        if(order_date > len(item_demand)-1):
            run_complete = True
            break
    tot_obj_val = tot_disposal +  tot_ordering + tot_fixed + tot_max_order     
    return (orders, tot_obj_val, tot_disposal, tot_ordering, tot_fixed, tot_max_order)

obj_val = 0 
vol = 0
tot_disposal = 0
tot_fixed = 0
tot_ordering = 0
tot_max_order = 0

schedule = {}
for item in demand.keys():
    orders, val, disposal, ordering, fixed, max_order_p = silverMeal(item) 
    obj_val += val
    tot_disposal += disposal
    tot_fixed += fixed
    tot_ordering += ordering
    tot_max_order += max_order_p
    for t in orders.keys():
        if orders[t] == 0:
            continue
        if item not in schedule.keys():
            schedule[item] = {}
        if t not in schedule[item].keys():
            schedule[item][t] = 0
        schedule[item][t] += orders[t]
        
    
    
#print(silverMeal('00ee8964'))
print(obj_val)
print('Ordering',tot_ordering)
print('Fixing ', tot_fixed)
print('Clearance ', tot_disposal)
print('Max Order ', tot_max_order)


00027afc
5.0
00084396
4.0
0025253f
3.0
00299fa6
002b8d7d
4.0
0034e982
8.0
00415c43
0057e9b7
2.0
0059a703
1.0
0062b624
1.0
00718852
0072bade
3.0
007d0c15
0098d2e1
3.0
009aa26d
00c31bc1
00d09a16
00e5e5d4
00f7bc48
8.0
00f95978
3.0
010001ce
3.0
01039c78
0103bbf4
1.0
0105db71
8.0
010a9e81
011f544e
01527ae0
14.0
01581e6a
01594a1e
1.0
015bf73f
016d54fd
5.0
0173b075
2.0
018a81b7
5.0
018afad9
1.0
019b9022
01b8b1c9
1.0
01bfcb2c
01c1f8cb
01c4e30e
01c5bf11
01cac4fc
01cb9c42
01ea7fcd
01ed569a
01f22aa0
4.0
01f433bd
3.0
01f68856
3.0
02031c33
0220bc95
022168b8
2.0
022d26e7
3.0
022dfc45
02332f98
1.0
0235666f
023d653f
02419907
6.0
02482fda
02483441
6.0
024eae4b
0257ffc4
1.0
02622c40
8.0
026f10bb
02705efe
027964bd
2.0
027d4d48
3.0
0298d33b
1.0
02a345a4
02a45767
02a6e04b
02a94211
6.0
02b46d8c
02c50464
3.0
02d50e38
02d9d785
02e0548c
02e99338
4.0
02ea5871
2.0
02f03050
9.0
02f56e8e
4.0
02fe252a
1.0
030f80b4
1.0
0315154e
0316ff46
031eb3e4
032285e5
6.0
0327934a
2.0
0331a828
2.0
03397693
3.0
03399708
3.0
034024

802540.2499999999
Ordering 672191.3999999996
Fixing  106309.69999999995
Clearance  15579.59999999999
Max Order  8459.55

In [396]:
buf_cost = 0
vol = 0
time_indexes
volumes = {}

for time in range(len(time_indexes)):
    for item in schedule:
        if time in schedule[item].keys():
            vol += schedule[item][time] * cu_per_tu[item] * volume_per_cu[item]

    if time > 0:
        for item in demand.keys():
            if time in demand[item].keys():
                vol -= demand[item][time] * volume_per_cu[item]

    volumes[time] = vol
    if vol > warehouse_volume:

        buf_cost += (vol - warehouse_volume) * buffer_cost

buf_cost
obj_val = obj_val + buf_cost

In [397]:
cost_dict['total'] = obj_val
cost_dict['buffer'] = buf_cost
cost_dict['clearance'] = tot_disposal
cost_dict['fixed'] = tot_fixed
cost_dict['per_tu'] = tot_ordering
cost_dict['max_order_penalty'] = tot_max_order
cost_dict

{'fixed': 3926277.5000000093,
 'per_tu': 41004819.90000003,
 'clearance': 27602.9,
 'buffer': 1681551.5886230997,
 'max_order_penalty': 656028.8999999997,
 'total': 47296280.78862308}

In [398]:
obj_val

47296280.78862308

In [399]:
schedule

{'00027afc': {0: 4,
  3: 8,
  11: 5,
  16: 6,
  23: 2,
  25: 6,
  31: 7,
  38: 6,
  46: 6,
  53: 3,
  58: 2,
  63: 4,
  66: 8,
  78: 12,
  93: 3,
  99: 1,
  113: 3,
  121: 4,
  127: 12,
  142: 4,
  148: 3,
  151: 2,
  155: 4,
  162: 2,
  166: 4,
  169: 23,
  193: 2,
  197: 1,
  200: 1,
  204: 4,
  211: 5,
  221: 3,
  225: 6,
  231: 14,
  234: 25,
  256: 6,
  268: 2,
  274: 2,
  277: 2,
  282: 1,
  288: 2,
  292: 1,
  296: 2,
  301: 3,
  303: 19,
  326: 1,
  330: 1,
  336: 2,
  340: 1,
  344: 2,
  351: 1,
  354: 1,
  360: 1},
 '00084396': {0: 1,
  4: 1,
  9: 1,
  16: 1,
  18: 2,
  23: 2,
  26: 1,
  30: 1,
  36: 1,
  38: 1,
  42: 1,
  49: 1,
  52: 1,
  57: 1,
  60: 1,
  64: 1,
  68: 1,
  73: 1,
  78: 1,
  81: 1,
  85: 1,
  87: 1,
  89: 1,
  95: 1,
  99: 1,
  101: 1,
  103: 1,
  106: 1,
  108: 1,
  112: 1,
  114: 1,
  119: 1,
  126: 1,
  129: 1,
  136: 1,
  141: 1,
  144: 1,
  149: 1,
  156: 1,
  158: 1,
  163: 1,
  171: 1,
  177: 1,
  182: 5,
  185: 2,
  186: 6,
  197: 1,
  199: 1,
  204

In [400]:
def neighbourhoodOperator1(schedule, obj_val):
    max_improvement = math.inf
    cost = obj_val
    modified_schedule = {}
    day_wise_orders = {}

    buffer_penalty_tot = 0
    max_order_penalty_tot = 0
    fixed_ordering_cost_tot = 0 
    schedule_imp = schedule 
    count = 0
    iterator = 0
    MAX_ITER = 100
    MAX_REPEAT = 3
    prev_max_improvement = 0

    while count < MAX_REPEAT and iterator < MAX_ITER:
        for item in schedule_imp:
            buffer_penalty = 0
            max_order_penalty = 0 
            cost_improvement = 0
            modified_orders = {}
            # Check if the item has a second order date
            if len(schedule_imp[item]) > 1:
                order_days = list(schedule_imp[item].keys())
                for i in range (0,len(order_days)-1, 2):
                    day1 = i
                    day2 = i + 1
                    if order_days[day2] - order_days[day1] <= shelf_life[item]:
                        tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
                        vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
                        if vol + volumes[day1] > warehouse_volume:
                            buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
                        if tot_orders > maximum_order_quantity_tu[item]:
                            max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
                        delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
                        if delta < 0:
                            fixed_ordering_cost_tot += ordering_cost_fixed[item] 
                            buffer_penalty_tot += buffer_penalty
                            max_order_penalty_tot += max_order_penalty
                            cost += delta
                            modified_orders[order_days[day1]] = tot_orders
                            volumes[order_days[day1]] += vol
                        else:
                            modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
                            modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
                modified_schedule[item] = modified_orders
            
            else:
                if len(schedule[item]) == 1 : 
                    modified_schedule[item] = schedule_imp[item]
                else:
                    continue


            if cost!=0 and cost < max_improvement:
                max_improvement = cost
        
        print("Buffer penalty paid", buffer_penalty_tot)
        print("max order penalty paid", max_order_penalty_tot)
        print("fixed cost reduced by", fixed_ordering_cost_tot)
        schedule_imp = modified_schedule
        obj_val = max_improvement
        iterator +=1

        print(max_improvement)

        if(max_improvement==prev_max_improvement):
            count+=1
        prev_max_improvement = max_improvement
    
    return schedule, obj_val, buffer_penalty_tot, max_order_penalty_tot, fixed_ordering_cost_tot

In [401]:
schedule, obj_val, buffer_cost_delta, max_order_penalty_delta, fixed_cost_delta = neighbourhoodOperator1(schedule, obj_val)
cost_dict['buffer'] += buffer_cost_delta
cost_dict['max_order_penalty'] += max_order_penalty_delta
cost_dict['fixed'] -= fixed_cost_delta
cost_dict['total'] = obj_val
cost_dict

Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 868942.0999999949
46433234.71181685
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871300.6999999959
46430876.11181691
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871694.0999999956
46430482.71181689
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871720.1999999955
46430456.61181689
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871720.1999999955
46430456.61181689
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871720.1999999955
46430456.61181689
Buffer penalty paid 5493.423195969735
max order penalty paid 402.59999999999997
fixed cost reduced by 871720.1999999955
46430456.61181689


{'fixed': 3054557.300000014,
 'per_tu': 41004819.90000003,
 'clearance': 27602.9,
 'buffer': 1687045.0118190695,
 'max_order_penalty': 656431.4999999997,
 'total': 46430456.61181689}

In [402]:
new_clearance_cost = 0
ordering_cost_per_tu_saved = 0

# Add clearance cost
# If I ever have to throw away equal to or more than a TU's worth, then I can just reduce order
def removeExpired(inv,life,item):
    global ordering_cost_per_tu_saved
    global new_clearance_cost
    
    tu_saved = 0
    # if inventory has items from days which are now too old
    if(len(inv)>life):
        amount_cleared = inv[0]

        tu_saved = amount_cleared//cu_per_tu[item]
        # ordering cost reduced here
        ordering_cost_per_tu_saved += tu_saved * ordering_cost_per_tu[item]

        amount_inevitably_cleared = amount_cleared%cu_per_tu[item]
        # clearance cost recalculated here
        new_clearance_cost += amount_inevitably_cleared*clearing_cost_per_cu[item]
        
        # remove days which are now too old
        inv = inv[-life:]
    
    return inv, tu_saved

def addDay(inv,ordered):
    inv.append(ordered)
    return inv

# how much of the current demand can be satisfied from items in inventory
def satisfyFromInventory(inv, dem):
    for day in range(len(inv)):
        if(dem==0):
            break
        if(inv[day] <= dem):
            dem -= inv[day]
            inv[day] = 0
        else:
            inv[day] -= dem
            dem = 0
    return inv

def updateInventory(inv,item,t,ordered):
    life = shelf_life[item]
    dem = demand[t][item]
    # first the orders of the new day are added to the inventory
    # and then the total inventory is used to satisfy the demand
    # This would mean that the demand cannot go unsatisfied from
    # the items in the inventory
    inv = addDay(inv, ordered)
    inv, tu_saved = removeExpired(inv,life,item)
    inv = satisfyFromInventory(inv, dem)
    return inv, tu_saved

def calculateMaxOrderPenalty(schedule):
    max_order_penalty = 0
    for day in schedule.keys():
        for item in schedule[day].keys():
            if(schedule[day][item] > maximum_order_quantity_tu[item]):
                max_order_penalty += (schedule[day][item] - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
    return max_order_penalty

def neighbourhoodOperator2(schedule):
    # dictionary of format {'item':['q left on earliest possible','q left on earliest possible + 1',...,'q left today']}
    inventory = {}

    for t in time_indexes:
        for item in articles:

            if item not in inventory.keys():
                inventory[item] = []

            # if there was no order for an item in a particular day, then the order was 0
            if(t in schedule.keys() and item in schedule[t].keys()):
                ordered = schedule[t][item] * cu_per_tu[item]
            else:
                ordered = 0

            # update the inventory according to what has been ordered today for item
            inventory[item],tu_saved = updateInventory(inventory[item], item, t, ordered)
            
            # Removing excess TUs from the schedule. On day t, we remove the excess from day t-shelf life
            if(t-shelf_life[item] in schedule.keys() and item in schedule[t-shelf_life[item]].keys()):
                if(t>=shelf_life[item]):
                    schedule[t-shelf_life[item]][item] -= tu_saved

                
    print("Clearance cost: ", new_clearance_cost)
    print("Ordering cost per TU saved: ", ordering_cost_per_tu_saved)

    new_max_order_penalty = calculateMaxOrderPenalty(schedule)

    return schedule, new_clearance_cost, ordering_cost_per_tu_saved, new_max_order_penalty

In [403]:
def transposeScheduleToDayFirst(schedule):
    result_dict = {}
    for day, articles in schedule.items():
        for article, quantity in articles.items():
            if article in result_dict:
                result_dict[article][day] = quantity
            else:
                result_dict[article] = {day: quantity}
    return result_dict

def transposeScheduleToItemFirst(schedule):
    result_dict = {}
    for article, days in schedule.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][article] = quantity
            else:
                result_dict[day] = {article: quantity}
    return result_dict

In [404]:
def transposeDemand(demand):
    result_dict = {}
    for item, days in demand.items():
        for day, quantity in days.items():
            if day in result_dict:
                result_dict[day][item] = quantity
            else:
                result_dict[day] = {item:quantity}
    return result_dict
demand = transposeDemand(demand)

In [405]:
schedule = transposeScheduleToDayFirst(schedule)
schedule, new_clearance_cost, per_tu_delta, new_max_order_penalty = neighbourhoodOperator2(schedule)
cost_dict['clearance'] = new_clearance_cost
cost_dict['max_order_penalty'] = new_max_order_penalty
cost_dict['per_tu'] -= per_tu_delta
cost_dict['total'] = cost_dict['fixed'] + cost_dict['per_tu'] + cost_dict['buffer'] + cost_dict['clearance'] + cost_dict['max_order_penalty']

Clearance cost:  41016.5
Ordering cost per TU saved:  1723240.4000000001


In [406]:
cost_dict

{'fixed': 3054557.300000014,
 'per_tu': 39281579.50000003,
 'clearance': 41016.5,
 'buffer': 1687045.0118190695,
 'max_order_penalty': 656028.900000004,
 'total': 44720227.21181912}

In [407]:
# max_improvement = math.inf
# cost = obj_val
# modified_schedule = {}
# day_wise_orders = {}

# buffer_penalty_tot = 0
# max_order_penalty_tot = 0
# fixed_ordering_cost_tot = 0 
# schedule_imp = schedule 
# count = 0
# iterator = 0
# MAX_ITER = 100
# MAX_REPEAT = 3
# prev_max_improvement = 0

# while count < MAX_REPEAT and iterator < MAX_ITER:
#     for item in schedule_imp:
#         buffer_penalty = 0
#         max_order_penalty = 0 
#         cost_improvement = 0
#         modified_orders = {}
#         # Check if the item has a second order date
#         if len(schedule_imp[item]) > 1:
#             order_days = list(schedule_imp[item].keys())
#             for i in range (0,len(order_days)-1, 2):
#                 day1 = i
#                 day2 = i + 1
#                 if order_days[day2] - order_days[day1] <= shelf_life[item]:
#                     tot_orders = schedule_imp[item][order_days[day1]]+ schedule_imp[item][order_days[day2]]
#                     vol = tot_orders * cu_per_tu[item] * volume_per_cu[item]
#                     if vol + volumes[day1] > warehouse_volume:
#                         buffer_penalty += buffer_cost * (vol + volumes[day1] - warehouse_volume)
#                     if tot_orders > maximum_order_quantity_tu[item]:
#                         max_order_penalty += ( tot_orders - maximum_order_quantity_tu[item]) * max_order_exceed_multiplier * ordering_cost_per_tu[item]
#                     delta = - ordering_cost_fixed[item] + buffer_penalty + max_order_penalty 
#                     if delta < 0:
#                         fixed_ordering_cost_tot += ordering_cost_fixed[item] 
#                         buffer_penalty_tot += buffer_penalty
#                         max_order_penalty_tot += max_order_penalty
#                         cost += delta
#                         modified_orders[order_days[day1]] = tot_orders
#                         volumes[order_days[day1]] += vol
#                     else:
#                         modified_orders[order_days[day1]] = schedule_imp[item][order_days[day1]]
#                         modified_orders[order_days[day2]] = schedule_imp[item][order_days[day2]]
#             modified_schedule[item] = modified_orders
        
#         else:
#             if len(schedule[item]) == 1 : 
#                 modified_schedule[item] = schedule_imp[item]
#             else:
#                 continue


#         if cost!=0 and cost < max_improvement:
#             max_improvement = cost
    
#     print("Buffer penalty paid", buffer_penalty_tot)
#     print("max order penalty paid", max_order_penalty_tot)
#     print("fixed cost reduced by", fixed_ordering_cost_tot)
#     schedule_imp = modified_schedule
#     obj_val = max_improvement
#     iterator +=1

#     print(max_improvement)

#     if(max_improvement==prev_max_improvement):
#         count+=1
#     prev_max_improvement = max_improvement